In [1]:
import tensorflow as tf
import numpy as np
import time
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from sklearn.metrics import classification_report

In [2]:
# Path SavedModel FP32 (baseline)
SAVED_MODEL_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40"

# Output TFLite FP16
TFLITE_FP16_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Final/pruned-mobilenetv3_large_fp16.tflite"

# Dataset test
TEST_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 1   # Konsisten untuk benchmarking inferensi

In [3]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = test_generator.num_classes

Found 383 images belonging to 6 classes.


In [4]:
# Proses Quantization PTQ FP16
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)

# Aktifkan optimisasi
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# FP16 quantization
converter.target_spec.supported_types = [tf.float16]

# Konversi
tflite_fp16_model = converter.convert()

# Simpan model
with open(TFLITE_FP16_PATH, "wb") as f:
    f.write(tflite_fp16_model)

In [5]:
# Ukuran Model
model_size_fp16 = os.path.getsize(TFLITE_FP16_PATH) / (1024 * 1024)
print(f"Ukuran model FP16: {model_size_fp16:.2f} MB")

Ukuran model FP16: 5.85 MB


In [6]:
# Load TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path=TFLITE_FP16_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
# Evaluasi Inferensi (Waktu & Prediksi)
y_true = []
y_pred = []

start_time = time.time()

for i in range(len(test_generator)):
    x_batch, y_batch = test_generator[i]

    interpreter.set_tensor(input_details[0]['index'], x_batch)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])

    y_true.append(np.argmax(y_batch))
    y_pred.append(np.argmax(output))

end_time = time.time()

In [8]:
# Laporan Waktu Inferensi
total_time = end_time - start_time
avg_inference_time = total_time / len(test_generator)

print("Time Report (FP16)")
print(f"Total inference time  : {total_time:.4f} detik")
print(f"Rata-rata per gambar  : {avg_inference_time:.6f} detik")

Time Report (FP16)
Total inference time  : 3.6152 detik
Rata-rata per gambar  : 0.009439 detik


In [9]:
# Classification Report
print("Classification Report (FP16)")
print(
    classification_report(
        y_true,
        y_pred,
        target_names=list(test_generator.class_indices.keys())
    )
)

Classification Report (FP16)
              precision    recall  f1-score   support

   cardboard       0.93      0.85      0.89        61
       glass       0.83      0.92      0.88        76
       metal       0.86      0.87      0.86        62
       paper       0.88      0.90      0.89        90
     plastic       0.83      0.82      0.83        72
       trash       0.82      0.64      0.72        22

    accuracy                           0.86       383
   macro avg       0.86      0.83      0.84       383
weighted avg       0.86      0.86      0.86       383

